In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

for p in ["..", "../..", "../../app"]:
    module_path = os.path.abspath(os.path.join(p))
    if module_path not in sys.path:
        sys.path.append(module_path)

import index

import app.elastic_query as elastic_query  # noqa: E402

In [ ]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "hammasir"),
    verify_certs=False,
    ssl_show_warn=False,
)
es.info()

In [ ]:
data = index.get_dat("../data/processed/base_dataset.csv")
index.index(data, reindex=False)

In [ ]:
# Example usage
search_params = {
    "city": ["مشهد"],
    # "gender": ["F"],
    "expertise": ["کمرم"],
}

query = elastic_query.build_query(search_params)
results = es.search(index="doctors", query=query)

for result in results["hits"]["hits"]:
    print(result)
    result = result["_source"]
    print(result["expertise"], result["star"])

len(results)

In [ ]:
from pprint import pprint

pprint(
    es.explain(
        index="doctors", id="14951", query=elastic_query.build_query(search_params)
    ).raw
)